In [6]:
from syncode.infer import Syncode

grammar = r"""
                // Lark grammar to validate single emoji output for TweetEval dataset
                start: emoji

                // Define the 3 emojis from the TweetEval emoji dataset
                emoji: "😍" | "😂" | "😉" 

                // Ensure there is no whitespace or other characters
                %import common.WS
                %ignore WS
        """

syn_llm = Syncode(model="meta-llama/Llama-3.1-8B-Instruct", grammar=grammar, new_mask_store=True, max_new_tokens=5)

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  7.98it/s]


[2025-03-28 22:42:15,809-syncode.mask_store.mask_store] - Using cache: False and fsm path cache/mask_stores/PreTrainedTokenizerFast/grammar_strict_2646269638_128000.pkl exist: False
[2025-03-28 22:42:15,809-syncode.mask_store.mask_store] - Creating mask store for PreTrainedTokenizerFast and custom, may take more than 10 minutes. Caching at /home/shubham/syncode/cache/mask_stores/PreTrainedTokenizerFast/grammar_strict_2646269638_128000.pkl.
[2025-03-28 22:42:16,417-syncode.mask_store.fsm_set] - 4 FSMs with 39 states initialized in 0.00 seconds
[2025-03-28 22:42:16,418-syncode.mask_store.mask_store] - Ignore whitespace is True
[2025-03-28 22:42:16,419-syncode.mask_store.mask_store] - Number of 2 length terminal sequences reduced from 16 to 4


100%|██████████| 39/39 [00:06<00:00,  5.83it/s]

[2025-03-28 22:42:23,118-syncode.mask_store.mask_store] - Time taken to create mask store: 7.31 seconds


In [7]:
prompt = """You are evaluating tweets to assign the most appropriate emoji. 

INSTRUCTIONS:
1. Read the tweet below carefully.
2. Select the SINGLE most appropriate emoji that captures the sentiment or topic.
3. Respond with ONLY that emoji - no words, explanations, or other characters.

The emoji must be one of the 3 valid options from this set:
😍 😂 😉

Example:
Tweet: "I love this new movie!"
Your response: 😍

Tweet: "This is so funny!"
Your response: 😂

Tweet: "{tweet_text}"
Your response:
"""

from datasets import load_dataset
dataset = load_dataset("tweet_eval", "emoji")

# Create mapping from integer labels to emoji characters
emoji_map = {
    1: "😍", 2: "😂", 14: "😉"
}

cnt_correct = 0

# Filter 100 examples from the test set where the label is 5 valid emojis 😍 😂 😉
examples = []
total_examples = 100

for i in range(len(dataset['test'])):
    label = dataset['test'][i]['label']
    if label in emoji_map:
        examples.append(dataset['test'][i])
        if len(examples) == total_examples:
            break

# Test the model with the first 20 examples
print("\nTesting the model with the first 20 examples:")

for i in range(total_examples):
    # Get the tweet text and label from the dataset
    tweet_text = examples[i]['text']
    label = examples[i]['label']
    
    # Map the integer label to the corresponding emoji
    expected_emoji = emoji_map[label]
    
    # Create the prompt with the tweet text
    prompt_with_tweet = prompt.format(tweet_text=tweet_text)
    
    # Generate the response using Syncode
    response = syn_llm.infer(prompt_with_tweet)
    generated_empoji = response[0].strip()[0]
    # except Exception as e:
    #     print(f"Error generating emoji for tweet {i}: {e}")
    #     generated_empoji = None

    # Print the results
    print(f"Tweet: {tweet_text}")
    print(f"Expected Emoji: {expected_emoji}")
    print(f"Generated Emoji: {generated_empoji}")

    if generated_empoji == expected_emoji:
        print("Correct!")
        cnt_correct += 1

    print("-" * 50)

# Print the total number of correct predictions
print(f"Total Correct Predictions: {cnt_correct} out of {total_examples}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Testing the model with the first 20 examples:
Tweet: en Pelham Parkway
Expected Emoji: 😂
Generated Emoji: 😂
Correct!
--------------------------------------------------
Tweet: This little lady is 26 weeks pregnant today! Excited for baby Cam to come! @ Springfield,…
Expected Emoji: 😍
Generated Emoji: 😍
Correct!
--------------------------------------------------
Tweet: Sunrise. @ Miami South Beach,Florida
Expected Emoji: 😍
Generated Emoji: 😂
--------------------------------------------------
Tweet: About to Tumble into Tuesday. #Jeep #JeepWave #jeeplife #Jeepbeef #jeepnation #JeepPorn…
Expected Emoji: 😂
Generated Emoji: 😂
Correct!
--------------------------------------------------
Tweet: It took a fall festival, but she's finally behind bars @user @ Duluth Fall Festival
Expected Emoji: 😉
Generated Emoji: 😂
--------------------------------------------------
Tweet: These escort cards by heartpapersoul for our lavender theme are so pretty! #coastsidecouture…
Expected Emoji: 😍
Generated Emo